___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

<h1><p style="text-align: center;">Data Analysis with Python <br>Project - 1</p><h1> - Traffic Police Stops <img src="https://docs.google.com/uc?id=17CPCwi3_VvzcS87TOsh4_U8eExOhL6Ki" class="img-fluid" alt="CLRSWY" width="200" height="100"> 

Does the ``gender`` of a driver have an impact on police behavior during a traffic stop? **In this chapter**, you will explore that question while practicing filtering, grouping, method chaining, Boolean math, string methods, and more!

***

## Examining traffic violations

Before comparing the violations being committed by each gender, you should examine the ``violations`` committed by all drivers to get a baseline understanding of the data.

In this exercise, you'll count the unique values in the ``violation`` column, and then separately express those counts as proportions.

> Before starting your work in this section **repeat the steps which you did in the previos chapter for preparing the data.** Continue to this chapter based on where you were in the end of the previous chapter.

In [1]:
# %run./01PreparingTheDataForAnalysis(Student)_new.ipynb

In [2]:
import pandas as pd
# ri= pd.read_csv('RI_cleaned.csv').sample(50000, random_state=101)
ri = pd.read_csv('RI_cleaned.csv', nrows=50000)

drop_columns=["state", "county_name", "county_fips", "fine_grained_location"]
ri.drop(drop_columns, axis='columns', inplace=True)

ri.dropna(subset=['driver_gender'], inplace=True)

ri['is_arrested'] = ri.is_arrested.astype('bool')

combined = ri.stop_date.str.cat(ri.stop_time, sep=' ')
ri['stop_datetime'] = pd.to_datetime(combined)
ri.drop(["stop_date", "stop_time"], axis='columns', inplace=True)

ri.set_index('stop_datetime', inplace=True)
ri.head()

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,location_raw,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,
2005-01-02 01:55:00,RI-2005-00001,Zone K1,600,M,1985.0,20.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1
2005-01-02 20:30:00,RI-2005-00002,Zone X4,500,M,1987.0,18.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4
2005-01-04 12:55:00,RI-2005-00004,Zone X4,500,M,1986.0,19.0,W,White,Equipment/Inspection Violation,Equipment,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-01-06 01:30:00,RI-2005-00005,Zone X4,500,M,1978.0,27.0,B,Black,Equipment/Inspection Violation,Equipment,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-01-12 08:05:00,RI-2005-00006,Zone X1,0,M,1973.0,32.0,B,Black,Call for Service,Other,False,NaN,NaN,False,Citation,False,30+ Min,True,False,Zone X1


**INSTRUCTIONS**

*   Count the unique values in the ``violation`` column, to see what violations are being committed by all drivers.
*   Express the violation counts as proportions of the total.

In [3]:
ri.dropna().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2001 entries, 2005-01-24 20:32:00 to 2006-08-08 22:22:00
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2001 non-null   object 
 1   location_raw        2001 non-null   object 
 2   police_department   2001 non-null   object 
 3   driver_gender       2001 non-null   object 
 4   driver_age_raw      2001 non-null   float64
 5   driver_age          2001 non-null   float64
 6   driver_race_raw     2001 non-null   object 
 7   driver_race         2001 non-null   object 
 8   violation_raw       2001 non-null   object 
 9   violation           2001 non-null   object 
 10  search_conducted    2001 non-null   bool   
 11  search_type_raw     2001 non-null   object 
 12  search_type         2001 non-null   object 
 13  contraband_found    2001 non-null   bool   
 14  stop_outcome        2001 non-null   object 
 15  is_arrested        

In [4]:
# Count the unique values in the violation column, to see what violations are being committed by all drivers.
ri.violation.unique()

array(['Speeding', 'Equipment', 'Other', 'Moving violation',
       'Registration/plates'], dtype=object)

In [5]:
ri.violation.value_counts(dropna = False)

Speeding               36111
Moving violation        6522
Equipment               3022
Registration/plates     1463
Other                    892
Name: violation, dtype: int64

In [6]:
# alternate
# ri.groupby("violation")["id"].count()

In [7]:
# Express the violation counts as proportions of the total.
ri.violation.value_counts(normalize = True)
# normalize = True denirse yüzde değer döndürür.

Speeding               0.752156
Moving violation       0.135847
Equipment              0.062945
Registration/plates    0.030473
Other                  0.018579
Name: violation, dtype: float64

In [8]:
# alternate
ri.groupby("violation")["id"].count()/ri["violation"].count()

violation
Equipment              0.062945
Moving violation       0.135847
Other                  0.018579
Registration/plates    0.030473
Speeding               0.752156
Name: id, dtype: float64

***

## Comparing violations by gender

The question we're trying to answer is whether male and female drivers tend to commit different types of traffic violations.

You'll first create a ``DataFrame`` for each gender, and then analyze the ``violations`` in each ``DataFrame`` separately.

**INSTRUCTIONS**

*   Create a ``DataFrame``, female, that only contains rows in which ``driver_gender`` is ``'F'``.
*   Create a ``DataFrame``, male, that only contains rows in which ``driver_gender`` is ``'M'``.
*   Count the ``violations`` committed by female drivers and express them as proportions.
*   Count the violations committed by male drivers and express them as proportions.

In [9]:
ri.driver_gender.value_counts(dropna=False)

M    34701
F    13309
Name: driver_gender, dtype: int64

In [10]:
# Create a DataFrame, female, that only contains rows in which driver_gender is 'F'
# Create a DataFrame, male, that only contains rows in which driver_gender is 'M'.
female = ri[ri.driver_gender == "F"]
male = ri[ri.driver_gender == "M"]

In [11]:
female.head()

,id,location_raw,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,
2005-02-24 01:20:00,RI-2005-00016,Zone X3,200,F,1983.0,22.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone X3
2005-03-14 10:00:00,RI-2005-00019,Zone K3,300,F,1984.0,21.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K3
2005-03-29 23:20:00,RI-2005-00026,Zone K3,300,F,1971.0,34.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3
2005-06-06 13:20:00,RI-2005-00035,Zone X4,500,F,1986.0,19.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-06-18 16:30:00,RI-2005-00037,Zone X4,500,F,1964.0,41.0,W,White,Other Traffic Violation,Moving violation,False,NaN,NaN,False,Arrest Driver,True,30+ Min,False,False,Zone X4


In [12]:
# Count the violations committed by female drivers and express them as proportions.
female.violation.value_counts(normalize=True)*100

Speeding               81.118040
Moving violation        9.903073
Equipment               4.560824
Registration/plates     2.757532
Other                   1.660530
Name: violation, dtype: float64

In [13]:
# Count the violations committed by male drivers and express them as proportions.
male.violation.value_counts(normalize=True)*100

Speeding               72.951788
Moving violation       14.996686
Equipment               6.959454
Registration/plates     3.158410
Other                   1.933662
Name: violation, dtype: float64

In [14]:
# alternate
ri.groupby(["driver_gender"])["violation"].value_counts(normalize=True,dropna=False)*100

driver_gender  violation          
F              Speeding               81.118040
               Moving violation        9.903073
               Equipment               4.560824
               Registration/plates     2.757532
               Other                   1.660530
M              Speeding               72.951788
               Moving violation       14.996686
               Equipment               6.959454
               Registration/plates     3.158410
               Other                   1.933662
Name: violation, dtype: float64

In [15]:
# alternate (yanlış kullanım)
ri.groupby(["driver_gender","violation"]).value_counts(normalize=True,dropna=False)

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'

***

## Comparing speeding outcomes by gender

When a driver is pulled over for speeding, many people believe that gender has an impact on whether the driver will receive a ticket or a warning. Can you find evidence of this in the dataset?

First, you'll create two ``DataFrames`` of drivers who were stopped for ``speeding``: one containing ***females*** and the other containing ***males***.

Then, for each **gender**, you'll use the ``stop_outcome`` column to calculate what percentage of stops resulted in a ``"Citation"`` (meaning a ticket) versus a ``"Warning"``.

**INSTRUCTIONS**

*   Create a ``DataFrame``, ``female_and_speeding``, that only includes female drivers who were stopped for speeding.
*   Create a ``DataFrame``, ``male_and_speeding``, that only includes male drivers who were stopped for speeding.
*   Count the **stop outcomes** for the female drivers and express them as proportions.
*   Count the **stop outcomes** for the male drivers and express them as proportions.

In [ ]:
# Create a DataFrame, female_and_speeding, that only includes female drivers who were stopped for speeding.
# Create a DataFrame, male_and_speeding, that only includes male drivers who were stopped for speeding.

female_and_speeding = ri[(ri.driver_gender == "F") & (ri.violation == "Speeding")]
male_and_speeding = ri[(ri.driver_gender == "M") & (ri.violation == "Speeding")]

In [ ]:
female_and_speeding.head()

In [ ]:
# Count the stop outcomes for the female drivers and express them as proportions.
female_and_speeding.stop_outcome.value_counts(normalize=True)*100

In [ ]:
# Count the stop outcomes for the male drivers and express them as proportions.
male_and_speeding.stop_outcome.value_counts(normalize=True)*100

In [18]:
# alternate with groupby and xs
ri.groupby(["violation","driver_gender"])["stop_outcome"].value_counts(dropna=False, normalize=True).xs("Speeding")*100

driver_gender  stop_outcome    
F              Citation            97.341608
               Warning              1.296777
               Arrest Driver        0.741015
               N/D                  0.361245
               Arrest Passenger     0.231567
               No Action            0.027788
M              Citation            95.729804
               Arrest Driver        2.622951
               Warning              1.066561
               N/D                  0.339720
               Arrest Passenger     0.201462
               No Action            0.039502
Name: stop_outcome, dtype: float64

***

## Calculating the search rate

During a traffic stop, the police officer sometimes conducts a search of the vehicle. In this exercise, you'll calculate the percentage of all stops that result in a vehicle search, also known as the **search rate**.

**INSTRUCTIONS**

*   Check the data type of ``search_conducted`` to confirm that it's a ``Boolean Series``.
*   Calculate the search rate by counting the ``Series`` values and expressing them as proportions.
*   Calculate the search rate by taking the mean of the ``Series``. (It should match the proportion of ``True`` values calculated above.)

In [ ]:
ri.info()

In [ ]:
# Check the data type of search_conducted to confirm that it's a Boolean Series.
ri.search_conducted.astype("bool").dtype

In [ ]:
# alternate
ri["search_conducted"].apply(bool).dtypes

In [ ]:
# Calculate the search rate by counting the Series values and expressing them as proportions.
ri.search_conducted.value_counts(normalize=True)*100

In [ ]:
# Calculate the search rate by taking the mean of the Series. (It should match the proportion of True values calculated above.)
ri.search_conducted.mean()*100

***

## Comparing search rates by gender

You'll compare the rates at which **female** and **male** drivers are searched during a traffic stop. Remember that the vehicle search rate across all stops is about **3.8%**.

First, you'll filter the ``DataFrame`` by gender and calculate the search rate for each group separately. Then, you'll perform the same calculation for both genders at once using a ``.groupby()``.

**INSTRUCTIONS 1/3**

*   Filter the ``DataFrame`` to only include **female** drivers, and then calculate the search rate by taking the mean of ``search_conducted``.

In [ ]:
ri[ri["driver_gender"]=="F"]["search_conducted"].mean()*100

**INSTRUCTIONS 2/3**

*   Filter the ``DataFrame`` to only include **male** drivers, and then repeat the search rate calculation.

In [ ]:
ri[ri["driver_gender"]=="M"]["search_conducted"].mean()*100

**INSTRUCTIONS 3/3**

*   Group by driver gender to calculate the search rate for both groups simultaneously. (It should match the previous results.)

In [16]:
ri.groupby(["driver_gender"])["search_conducted"].value_counts(normalize=True)*100

driver_gender  search_conducted
F              False               98.219250
               True                 1.780750
M              False               94.884874
               True                 5.115126
Name: search_conducted, dtype: float64

In [17]:
# alternate
ri.groupby(["driver_gender"])["search_conducted"].value_counts(normalize=True).xs(["F",True])*100

1.7807498685100307

In [ ]:
# alternate
ri.groupby(["driver_gender"])["search_conducted"].value_counts(normalize=True).xs(["M",True])*100

***

## Adding a second factor to the analysis

Even though the search rate for males is much higher than for females, it's possible that the difference is mostly due to a second factor.

For example, you might hypothesize that the search rate varies by violation type, and the difference in search rate between males and females is because they tend to commit different violations.

You can test this hypothesis by examining the search rate for each combination of gender and violation. If the hypothesis was true, you would find that males and females are searched at about the same rate for each violation. Find out below if that's the case!

**INSTRUCTIONS 1/2**

*   Use a ``.groupby()`` to calculate the search rate for each combination of gender and violation. Are males and females searched at about the same rate for each violation?

In [ ]:
ri.groupby(["driver_gender","violation"])["search_conducted"].value_counts(normalize=True)*100

**INSTRUCTIONS 2/2**

*   Reverse the ordering to group by violation before gender. The results may be easier to compare when presented this way.

In [ ]:
ri.groupby(["violation","driver_gender"]).search_conducted.mean()

***

## Counting protective frisks

During a vehicle search, the police officer may pat down the driver to check if they have a weapon. This is known as a ``"protective frisk."``

You'll first check to see how many times "Protective Frisk" was the only search type. Then, you'll use a string method to locate all instances in which the driver was frisked.

**INSTRUCTIONS**

*   Count the ``search_type`` values to see how many times ``"Protective Frisk"`` was the only search type.
*   Create a new column, frisk, that is ``True`` if ``search_type`` contains the string ``"Protective Frisk"`` and ``False`` otherwise.
*   Check the data type of frisk to confirm that it's a ``Boolean Series``.
*   Take the sum of frisk to count the total number of frisks.

In [ ]:
# Count the search_type values to see how many times "Protective Frisk" was the only search type.
ri[ri.search_type=="Protective Frisk"].shape[0]

In [ ]:
ri.search_type.value_counts(dropna=False)

In [ ]:
#Create a new column, frisk, that is ``True`` if ``search_type`` contains the string ``"Protective Frisk"`` and ``False`` otherwise.
import numpy as np
ri["frisk"]=["Protective Frisk" in i for i in ri.search_type.fillna("")]

In [ ]:
ri.frisk.value_counts()

In [ ]:
# Check the data type of frisk to confirm that it's a ``Boolean Series``.
print(ri.frisk.dtypes)

In [ ]:
# Take the sum of frisk to count the total number of frisks.
ri.frisk.sum()

***

## Comparing frisk rates by gender

You'll compare the rates at which female and male drivers are frisked during a search. Are males frisked more often than females, perhaps because police officers consider them to be higher risk?

Before doing any calculations, it's important to filter the ``DataFrame`` to only include the relevant subset of data, namely stops in which a search was conducted.

**INSTRUCTIONS**

*   Create a ``DataFrame``, searched, that only contains rows in which ``search_conducted`` is ``True``.
*   Take the mean of the frisk column to find out what percentage of searches included a frisk.
*   Calculate the frisk rate for each gender using a ``.groupby()``.

In [ ]:
# Create a ``DataFrame``, searched, that only contains rows in which ``search_conducted`` is ``True``.
searched=ri[ri.search_conducted==True]
searched.search_conducted.value_counts()

In [ ]:
# Take the mean of the frisk column to find out what percentage of searches included a frisk.
searched.frisk.mean()*100

In [ ]:
# Calculate the frisk rate for each gender using a ``.groupby()``.
searched.groupby(["driver_gender"]).frisk.mean()*100